In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import keys
google_key = keys["google_maps"]

In [2]:
#import the csv to work with
city_weather = pd.read_csv("weatherpy_database.csv")
city_weather

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
0,0,Mataura,NZ,-46.1927,168.8643,49.48,78,100,0.43,overcast clouds,2021-11-04 19:11:39
1,1,Isangel,VU,-19.5500,169.2667,74.05,87,89,14.34,overcast clouds,2021-11-04 19:11:39
2,2,Nikolskoye,RU,59.7035,30.7861,46.40,94,26,6.91,scattered clouds,2021-11-04 19:09:27
3,3,Kudat,MY,6.8837,116.8477,77.99,86,86,2.53,overcast clouds,2021-11-04 19:11:39
4,4,Deogarh,IN,21.5333,84.7333,67.84,74,4,2.08,clear sky,2021-11-04 19:11:40
...,...,...,...,...,...,...,...,...,...,...,...
677,677,Cizre,TR,37.3272,42.1903,69.39,49,0,5.53,clear sky,2021-11-04 19:28:55
678,678,Biloela,AU,-24.4167,150.5000,62.94,85,56,4.45,broken clouds,2021-11-04 19:28:55
679,679,Miri,MY,4.4148,114.0089,77.14,100,40,3.44,light rain,2021-11-04 19:28:55
680,680,Haines Junction,CA,60.7522,-137.5108,21.15,86,83,0.00,broken clouds,2021-11-04 19:28:56


In [3]:
# This block will create a dataframe for the citys that meet your temp criteria

# creating analysis variables
min_temp = float(input("What is the minimum temperature for your vacation --> "))
max_temp = float(input("What is the maximum temperature for your vacation --> "))

# Creating a new dataframe that will be used for further charting
preffered_cities = city_weather.loc[((city_weather["Max Temp"] >= min_temp) & (city_weather["Max Temp"] <= max_temp))]

#check that the min and max are within the specified range
#print(min(preffered_cities["Max Temp"]))
#print(max(preffered_cities["Max Temp"]))

What is the minimum temperature for your vacation --> 60
What is the maximum temperature for your vacation --> 85


In [4]:
# drop all null values and then check if there are any errors left
preffered_cities = preffered_cities.dropna()
preffered_cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 1 to 681
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City_ID      318 non-null    int64  
 1   City         318 non-null    object 
 2   Country      318 non-null    object 
 3   Lat          318 non-null    float64
 4   Lng          318 non-null    float64
 5   Max Temp     318 non-null    float64
 6   Humidity     318 non-null    int64  
 7   Cloudiness   318 non-null    int64  
 8   Wind Speed   318 non-null    float64
 9   Description  318 non-null    object 
 10  Date         318 non-null    object 
dtypes: float64(4), int64(3), object(4)
memory usage: 29.8+ KB


In [5]:
# create a list of hotels located close to each city
hotel_df = preffered_cities[["City", "Country", "Max Temp", "Lat", "Lng","Description"]].copy()
hotel_df["Hotel Name"] = ""
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_key
}

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try : 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError) : 
        print("Seems they aint got any none hotels in" + row["City"] + "  \_(0.0)_/")

Seems they aint got any none hotels inBirao  \_(0.0)_/
Seems they aint got any none hotels inBoende  \_(0.0)_/
Seems they aint got any none hotels inAdrar  \_(0.0)_/
Seems they aint got any none hotels inLahij  \_(0.0)_/
Seems they aint got any none hotels inBeloha  \_(0.0)_/
Seems they aint got any none hotels inMarzuq  \_(0.0)_/
Seems they aint got any none hotels inGarachine  \_(0.0)_/
Seems they aint got any none hotels inMitu  \_(0.0)_/
Seems they aint got any none hotels inSangueya  \_(0.0)_/
Seems they aint got any none hotels inThinadhoo  \_(0.0)_/
Seems they aint got any none hotels inBeroroha  \_(0.0)_/
Seems they aint got any none hotels inAguas Vermelhas  \_(0.0)_/
Seems they aint got any none hotels inLaela  \_(0.0)_/


In [6]:
# drop cities with no hotels located in the required radius
hotel_df = hotel_df.dropna()

# save the vacation spot list to a csv
hotel_df.to_csv("weatherpy_vacation.csv")
hotel_df

,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
1,Isangel,VU,74.05,-19.5500,169.2667,overcast clouds,Tanna Lodge
3,Kudat,MY,77.99,6.8837,116.8477,overcast clouds,Dream Garden Hotel
4,Deogarh,IN,67.84,21.5333,84.7333,clear sky,Amit Henej House
5,Rincon,US,66.16,32.2960,-81.2354,broken clouds,Days Inn Rincon
6,Vao,NC,68.70,-22.6667,167.4833,overcast clouds,Hotel Kou-Bugny
...,...,...,...,...,...,...,...
675,Uyuni,BO,67.10,-20.4597,-66.8250,scattered clouds,Hostal La Magia de Uyuni
677,Cizre,TR,69.39,37.3272,42.1903,clear sky,Divan Otel Cizre / Şırnak
678,Biloela,AU,62.94,-24.4167,150.5000,broken clouds,Biloela Silo Motor Inn
679,Miri,MY,77.14,4.4148,114.0089,light rain,"Mega Hotel, Miri"


In [7]:
# create info boxes for a heatmap

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather description</dt><dd>{Description}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [8]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
gmaps.configure(api_key=google_key)
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))